# Reviewing ECMWF hindcasts 

## Check for possibility of a readiness trigger with a longer lead time

The current forecast (Meteo France La Reunion) provides limited lead time and we should explore if we could use ECMWF to have a readiness signal. I would basically check some basic perfomance metrics vs lead time and decide with partners what is acceptable.

In [8]:
%load_ext jupyter_black

import os
from datetime import datetime
from pathlib import Path
import requests
from dateutil import rrule

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


### Getting dates for cyclones

### Downloading Hindcasts

In [9]:
save_dir = Path(os.environ["MOZ_HINDCAST_DIR"])
email = input("email: ")
pswd = input("password: ")
values = {"email": email, "passwd": pswd, "action": "login"}
login_url = "https://rda.ucar.edu/cgi-bin/login"
values = {
    "orcid_id": "0009-0005-2381-1687",
    "api_token": "3540ef15c154baec6a21fe2587b2",
    "action": "tokenlogin",
}
ret = requests.post(login_url, data=values)
if ret.status_code != 200:
    print("Bad Authentication")
    print(ret.text)
    exit(1)

In [10]:
start_date = datetime(2020, 9, 16, 0, 0, 0)
dspath = "https://rda.ucar.edu/datasets/ds330.3/"
date_list = rrule.rrule(
    rrule.HOURLY,
    dtstart=start_date,
    until=datetime.utcnow().date(),
    interval=12,
)
verbose = True
dspath = "https://data.rda.ucar.edu/ds330.3/"

In [15]:
for date in date_list:
    ymd = date.strftime("%Y%m%d")
    ymdhms = date.strftime("%Y%m%d%H%M%S")
    server = "test" if date < datetime(2008, 8, 1) else "prod"
    file = (
        f"ecmf/{date.year}/{ymd}/z_tigge_c_ecmf_{ymdhms}_"
        f"ifs_glob_{server}_all_glo.xml"
    )
    filename = dspath + file
    outfile = save_dir / "xml" / os.path.basename(filename)
    # Don't download if exists already
    if outfile.exists():
        if verbose:
            print(f"{file} already exists")
        continue
    req = requests.get(filename, cookies=ret.cookies, allow_redirects=True)
    if req.status_code != 200:
        if verbose:
            print(f"{file} invalid URL")
        continue
    if verbose:
        print(f"{file} downloading")
    open(outfile, "wb").write(req.content)

ecmf/2020/20200916/z_tigge_c_ecmf_20200916000000_ifs_glob_prod_all_glo.xml downloading


FileNotFoundError: [Errno 2] No such file or directory: 'G:\\Shared drives\\Predictive Analytics\\CERF Anticipatory Action\\General - All AA projects\\Data\\public\\exploration\\moz\\ecmwf_hindcast\\xml\\z_tigge_c_ecmf_20200916000000_ifs_glob_prod_all_glo.xml'